In [20]:
import dask.dataframe as dd
df = dd.read_csv('s3://sar-store/plates/xaa.csv',storage_options={'anon': True})

In [144]:
import boto3
import os 
s3 = boto3.resource('s3', aws_access_key_id = os.environ['ACCESS'], aws_secret_access_key=os.environ['SECRET'])
my_bucket = s3.Bucket('sar-store')


    
paths = ["s3://" + object.bucket_name + '/' + object.key for object in my_bucket.objects.filter(Prefix='plates/')]
paths[1:]

['s3://sar-store/plates/xaa.csv',
 's3://sar-store/plates/xab1.csv',
 's3://sar-store/plates/xac2.csv',
 's3://sar-store/plates/xad.csv',
 's3://sar-store/plates/xae.csv',
 's3://sar-store/plates/xaf.csv',
 's3://sar-store/plates/xag.csv',
 's3://sar-store/plates/xah.csv',
 's3://sar-store/plates/xai.csv',
 's3://sar-store/plates/xaj.csv',
 's3://sar-store/plates/xak.csv',
 's3://sar-store/plates/xal.csv',
 's3://sar-store/plates/xam.csv',
 's3://sar-store/plates/xan.csv',
 's3://sar-store/plates/xao.csv',
 's3://sar-store/plates/xap.csv',
 's3://sar-store/plates/xaq.csv',
 's3://sar-store/plates/xar.csv',
 's3://sar-store/plates/xas.csv',
 's3://sar-store/plates/xat.csv',
 's3://sar-store/plates/xau.csv',
 's3://sar-store/plates/xav.csv',
 's3://sar-store/plates/xaw.csv',
 's3://sar-store/plates/xax.csv',
 's3://sar-store/plates/xay.csv',
 's3://sar-store/plates/xaz.csv',
 's3://sar-store/plates/xba.csv',
 's3://sar-store/plates/xbb.csv',
 's3://sar-store/plates/xbc.csv',
 's3://sar-s

In [145]:
df = dd.read_csv(paths[1:], storage_options={'anon': True}, header=None, names=col_names)

In [147]:
df.head()

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (3,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


,Plate,State,License Type,Summons Number,Issue Date,Violation Time,Violation,Judgment Entry Date,Fine Amount,Penalty Amount,Interest Amount,Reduction Amount,Payment Amount,Amount Due,Precinct,County,Issuing Agency,Violation Status,Summons Image
0,Plate,State,License Type,Summons Number,Issue Date,Violation Time,Violation,Judgment Entry Date,Fine Amount,Penalty Amount,Interest Amount,Reduction Amount,Payment Amount,Amount Due,Precinct,County,Issuing Agency,Violation Status,Summons Image
1,EHSD79,FL,PAS,8444539855,12/10/2016,03:23P,EXPIRED MUNI METER,NaN,65.00,10.00,0.00,0.00,75.00,0.00,009,NY,TRAFFIC,NaN,View Summons (http://nycserv.nyc.gov/NYCServWe...
2,GZT8961,NY,PAS,8489403065,12/24/2016,09:22A,NO PARKING-STREET CLEANING,NaN,45.00,10.00,0.00,0.00,55.00,0.00,102,Q,TRAFFIC,NaN,View Summons (http://nycserv.nyc.gov/NYCServWe...
3,DPJ6112,NY,PAS,8444539946,12/12/2016,06:57A,NO STANDING-BUS STOP,NaN,115.00,0.00,0.00,0.00,115.00,0.00,001,NY,TRAFFIC,HEARING HELD-GUILTY,View Summons (http://nycserv.nyc.gov/NYCServWe...
4,88810JY,NY,COM,8474261156,12/02/2016,07:31A,FAIL TO DISP. MUNI METER RECPT,NaN,65.00,30.00,0.00,95.00,0.00,0.00,014,NY,TRAFFIC,HEARING HELD-NOT GUILTY,View Summons (http://nycserv.nyc.gov/NYCServWe...


In [ ]:
len(df)

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (3,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)
/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)
